In [1]:
import GetPropertiesAPI as GP
import importlib
importlib.reload(GP)
import csv
import json
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [27]:
with open("../data/all_imgs_expt2.csv","r") as allImgs:
    gids = [row[0] for row in csv.reader(allImgs)]

gids = list(map(lambda x : int(x),gids))
datetimes = GP.getExifData(gids,'unixtime')
lats = GP.getExifData(gids,'lat')
longs = GP.getExifData(gids,'lon')

In [35]:
imgProps = {gids[i] : {'datetime' : GP.getUnixTimeReadableFmt(datetimes[i]),
                      'lat' : lats[i],
                      'long' : longs[i]} 
           for i in range(0,len(gids))}

with open("../data/imgs_exif_data.json","w") as outFl:
    json.dump(imgProps,outFl,indent=4)

In [2]:
with open("../data/imgs_exif_data.json","r") as inFl:
    imgProps = json.load(inFl)

In [3]:
gidExifDf = pd.DataFrame(imgProps).transpose()
gidExifDf.reset_index(inplace=True)
gidExifDf.columns = ['GID','datetime','lat','long']
gidExifDf[['GID']] = gidExifDf[['GID']].apply(pd.to_numeric)

In [4]:
with open("../FinalResults/rankListImages_expt2.csv","r") as inFl:
    gidResultsDf = pd.DataFrame.from_csv(inFl)
    
gidResultsDf.reset_index(inplace=True)

In [5]:
df = pd.merge(gidExifDf,gidResultsDf,left_on='GID',right_on='GID')

In [6]:
d = df.to_dict()['Proportion']
bin_shr = {}
for key in d.keys():
    if d[key] <= 20:
        bin_shr[key] = 0
    elif d[key] >= 80:
        bin_shr[key] = 1
        
D = df.to_dict()
D['bin_shr'] = bin_shr
df = pd.DataFrame.from_dict(D)

unshared = df[(df['bin_shr'] == 0)]
unshared = unshared[['lat','long']]

shared = df[(df['bin_shr'] == 1)]
shared = shared[['lat','long']]

In [8]:
shared_lats = list(shared['lat'])
shared_longs = list(shared['long'])

map_loc = folium.Map(location=[-1.35137,36.8003],tiles='Stamen Terrain',zoom_start=12)

for i in range(0,len(shared_lats)):
    folium.Marker([shared_lats[i],shared_longs[i]],
              icon=folium.Icon(color='green',icon='info-sign')
              ).add_to(map_loc)

unshared_lats = list(unshared['lat'])
unshared_longs = list(unshared['long'])

for i in range(0,len(unshared_lats)):
    folium.Marker([unshared_lats[i],unshared_longs[i]],
              icon=folium.Icon(color='red',icon='info-sign')
              ).add_to(map_loc)
    
map_loc.save(outfile='../FinalResults/ClickLocations.html')

In [9]:
cluster_map = folium.Map(location=[-1.35137,36.8003],tiles='Stamen Terrain',zoom_start=11)
marker_cluster = folium.MarkerCluster("Shares").add_to(cluster_map)
marker_cluster2 = folium.MarkerCluster("Not Shares").add_to(cluster_map)


for i in range(0,len(shared_lats)):
    folium.Marker([shared_lats[i],shared_longs[i]],
                  icon=folium.Icon(color='green',icon='info-sign'),
                  popup="Shared"
                 ).add_to(marker_cluster)

for i in range(0,len(unshared_lats)):
    folium.Marker([unshared_lats[i],unshared_longs[i]],
                 icon=folium.Icon(color='red',icon='info-sign'),
                  popup="Not Shared"
                 ).add_to(marker_cluster)
    
cluster_map.save(outfile='../FinalResults/ShareNoShareLocationClusters.html')